# Baseado no artigo Sentiment Analysis With Comparison Enhanced Deep Neural Network
# DOI: 10.1109/ACCESS.2020.2989424
# https://keras.io/examples/nlp/bidirectional_lstm_imdb/

In [26]:
import numpy as np
import keras
from keras import layers
import tensorflow as tf

# Obtenção dos dados

In [27]:
max_features = 20000  # Only consider the top 20k words
maxlen = 200  # Only consider the first 200 words of each movie review

(x_train, y_train), (x_val, y_val) = keras.datasets.imdb.load_data(
    num_words=max_features
)
print(len(x_train), "Training sequences")
print(len(x_val), "Validation sequences")

# Use pad_sequence to standardize sequence length:
# this will truncate sequences longer than 200 words and zero-pad sequences shorter than 200 words.
x_train = keras.utils.pad_sequences(x_train, maxlen=maxlen)
x_val = keras.utils.pad_sequences(x_val, maxlen=maxlen)

y_train = np.asarray(y_train).astype('int32').reshape((-1,1))
y_val = np.asarray(y_val).astype('int32').reshape((-1,1))

25000 Training sequences
25000 Validation sequences


# RNN

In [28]:
# Input for variable-length sequences of integers
inputs = keras.Input(shape=(maxlen,), dtype="int32", name="input")
# Embed each integer in a 128-dimensional vector
x = layers.Embedding(max_features, 128, name="embedding")(inputs)
# Add 1 bidirectional LSTMs
x = layers.SimpleRNN((64), name="rnn")(x)#
# Dropout layer
x = layers.Dropout(0.2)(x)
# Add a classifier
outputs = layers.Dense(1, activation="sigmoid", name="output")(x)
model = keras.Model(inputs, outputs)
model.summary()

Model: "model_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 200)]             0         
                                                                 
 embedding (Embedding)       (None, 200, 128)          2560000   
                                                                 
 rnn (SimpleRNN)             (None, 64)                12352     
                                                                 
 dropout_2 (Dropout)         (None, 64)                0         
                                                                 
 output (Dense)              (None, 1)                 65        
                                                                 
Total params: 2572417 (9.81 MB)
Trainable params: 2572417 (9.81 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [29]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy","Precision","AUC"])
model.fit(x_train, y_train, batch_size=32, epochs=2, validation_data=(x_val, y_val))

Epoch 1/2
782/782 [==============================] - 195s 247ms/step - loss: 0.6561 - accuracy: 0.6048 - precision: 0.6063 - auc: 0.6553 - val_loss: 0.6137 - val_accuracy: 0.6446 - val_precision: 0.6320 - val_auc: 0.7169
Epoch 2/2
782/782 [==============================] - 167s 214ms/step - loss: 0.5362 - accuracy: 0.7304 - precision: 0.7270 - auc: 0.8058 - val_loss: 0.5453 - val_accuracy: 0.7258 - val_precision: 0.7110 - val_auc: 0.7974


# Bi-LSTM

In [30]:
# Input for variable-length sequences of integers
inputs = keras.Input(shape=(maxlen,), dtype="int32", name="input")
# Embed each integer in a 128-dimensional vector
x = layers.Embedding(max_features, 128, name="embedding")(inputs)
# Add 1 bidirectional LSTMs
x = layers.Bidirectional(layers.LSTM(64, return_sequences=False), name="bilstm")(x)
# Dropout layer
x = layers.Dropout(0.2)(x)
# Add a classifier
outputs = layers.Dense(1, activation="sigmoid", name="output")(x)
model2 = keras.Model(inputs, outputs)
model2.summary()

Model: "model_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 200)]             0         
                                                                 
 embedding (Embedding)       (None, 200, 128)          2560000   
                                                                 
 bilstm (Bidirectional)      (None, 128)               98816     
                                                                 
 dropout_3 (Dropout)         (None, 128)               0         
                                                                 
 output (Dense)              (None, 1)                 129       
                                                                 
Total params: 2658945 (10.14 MB)
Trainable params: 2658945 (10.14 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [31]:
model2.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy","Precision","AUC"])
model2.fit(x_train, y_train, batch_size=32, epochs=2, validation_data=(x_val, y_val))

Epoch 1/2
782/782 [==============================] - 56s 66ms/step - loss: 0.4445 - accuracy: 0.7901 - precision: 0.7808 - auc: 0.8745 - val_loss: 0.3351 - val_accuracy: 0.8592 - val_precision: 0.8917 - val_auc: 0.9342
Epoch 2/2
782/782 [==============================] - 27s 34ms/step - loss: 0.2318 - accuracy: 0.9094 - precision: 0.9097 - auc: 0.9666 - val_loss: 0.3286 - val_accuracy: 0.8652 - val_precision: 0.8669 - val_auc: 0.9364


# Bi-LSTM empilhado

In [32]:
# Input for variable-length sequences of integers
inputs = keras.Input(shape=(maxlen,), dtype="int32", name="input")
# Embed each integer in a 128-dimensional vector
x = layers.Embedding(max_features, 128, name="embedding")(inputs)
# Add 2 bidirectional LSTMs
x = layers.Bidirectional(layers.LSTM(64, return_sequences=True), name="bilstm1")(x)
x = layers.Bidirectional(layers.LSTM(64, return_sequences=False), name="bilstm2")(x)
# Dropout layer
x = layers.Dropout(0.2)(x)
# Add a classifier
outputs = layers.Dense(1, activation="sigmoid", name="output")(x)
model3 = keras.Model(inputs, outputs)
model3.summary()

Model: "model_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 200)]             0         
                                                                 
 embedding (Embedding)       (None, 200, 128)          2560000   
                                                                 
 bilstm1 (Bidirectional)     (None, 200, 128)          98816     
                                                                 
 bilstm2 (Bidirectional)     (None, 128)               98816     
                                                                 
 dropout_4 (Dropout)         (None, 128)               0         
                                                                 
 output (Dense)              (None, 1)                 129       
                                                                 
Total params: 2757761 (10.52 MB)
Trainable params: 2757761

In [33]:
model3.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy","Precision","AUC"])
model3.fit(x_train, y_train, batch_size=32, epochs=2, validation_data=(x_val, y_val))

Epoch 1/2
782/782 [==============================] - 72s 84ms/step - loss: 0.4026 - accuracy: 0.8205 - precision: 0.8176 - auc: 0.8986 - val_loss: 0.3692 - val_accuracy: 0.8436 - val_precision: 0.8160 - val_auc: 0.9231
Epoch 2/2
782/782 [==============================] - 38s 49ms/step - loss: 0.2655 - accuracy: 0.8964 - precision: 0.8994 - auc: 0.9561 - val_loss: 0.3536 - val_accuracy: 0.8472 - val_precision: 0.8928 - val_auc: 0.9301


# MultiHead-Attention

In [34]:
# Input for variable-length sequences of integers
inputs = keras.Input(shape=(maxlen,), dtype="int32", name="input")
# Embed each integer in a 128-dimensional vector
x = layers.Embedding(max_features, 128, name="embedding")(inputs)
# Add a Multi-Head Attention layer
x = layers.MultiHeadAttention(8, 64, 64, name="mha")(x,x)
# Dropout layer
x = layers.Dropout(0.2)(x)
# Add a classifier
outputs = layers.Dense(1, activation="sigmoid", name="output")(x)
outputs = tf.reduce_mean(outputs, axis=1)
model4 = keras.Model(inputs, outputs)
model4.summary()

Model: "model_11"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input (InputLayer)          [(None, 200)]                0         []                            
                                                                                                  
 embedding (Embedding)       (None, 200, 128)             2560000   ['input[0][0]']               
                                                                                                  
 mha (MultiHeadAttention)    (None, 200, 128)             263808    ['embedding[0][0]',           
                                                                     'embedding[0][0]']           
                                                                                                  
 dropout_5 (Dropout)         (None, 200, 128)             0         ['mha[0][0]']          

In [35]:
model4.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy","Precision","AUC"])
model4.fit(x_train, y_train, batch_size=32, epochs=2, validation_data=(x_val, y_val))

Epoch 1/2
782/782 [==============================] - 52s 64ms/step - loss: 0.3483 - accuracy: 0.8375 - precision: 0.8320 - auc: 0.9249 - val_loss: 0.2962 - val_accuracy: 0.8757 - val_precision: 0.8807 - val_auc: 0.9467
Epoch 2/2
782/782 [==============================] - 29s 37ms/step - loss: 0.1792 - accuracy: 0.9304 - precision: 0.9277 - auc: 0.9800 - val_loss: 0.3812 - val_accuracy: 0.8518 - val_precision: 0.9013 - val_auc: 0.9361


# Bi-LSTM com MultiHead-Attention

In [36]:
# Input for variable-length sequences of integers
inputs = keras.Input(shape=(maxlen,), dtype="int32", name="input")
# Embed each integer in a 128-dimensional vector
x = layers.Embedding(max_features, 128, name="embedding")(inputs)
# Add 1 bidirectional LSTMs
x = layers.Bidirectional(layers.LSTM(64, return_sequences=True), name="bilstm")(x)
# Add a Multi-Head Attention layer
x = layers.MultiHeadAttention(8, 64, name="mha")(x,x)
# Dropout layer
x = layers.Dropout(0.2)(x)
# Add a classifier
outputs = layers.Dense(1, activation="sigmoid", name="output")(x)
outputs = tf.reduce_mean(outputs, axis=1)
model5 = keras.Model(inputs, outputs)
model5.summary()

Model: "model_12"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input (InputLayer)          [(None, 200)]                0         []                            
                                                                                                  
 embedding (Embedding)       (None, 200, 128)             2560000   ['input[0][0]']               
                                                                                                  
 bilstm (Bidirectional)      (None, 200, 128)             98816     ['embedding[0][0]']           
                                                                                                  
 mha (MultiHeadAttention)    (None, 200, 128)             263808    ['bilstm[0][0]',              
                                                                     'bilstm[0][0]']       

In [37]:
model5.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy","Precision","AUC"])
model5.fit(x_train, y_train, batch_size=32, epochs=2, validation_data=(x_val, y_val))

Epoch 1/2
782/782 [==============================] - 69s 81ms/step - loss: 0.3425 - accuracy: 0.8412 - precision: 0.8481 - auc: 0.9280 - val_loss: 0.2697 - val_accuracy: 0.8858 - val_precision: 0.8910 - val_auc: 0.9562
Epoch 2/2
782/782 [==============================] - 38s 49ms/step - loss: 0.1528 - accuracy: 0.9429 - precision: 0.9410 - auc: 0.9852 - val_loss: 0.3176 - val_accuracy: 0.8756 - val_precision: 0.8798 - val_auc: 0.9472
